In [14]:
#multiSVM
import numpy as np

In [15]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [16]:

label = unpickle("CS231n/cifar-10-batches-py/batches.meta")
dict = unpickle("CS231n/cifar-10-batches-py/data_batch_2")

In [17]:
dict.keys()

dict_keys([b'batch_label', b'labels', b'data', b'filenames'])

In [18]:
trainX = dict[b'data']
trainY = dict[b'labels']

In [45]:
lam = 1e-5
delta = 1
learning_rate = 1e-4
classification = 10
w = 0.001 * np.random.randn(trainX.shape[1], classification)
w.shape


(3072, 10)

In [22]:
def predict(w):
    dictTest = unpickle("CS231n/cifar-10-batches-py/data_batch_5")
    testX = dictTest[b'data']
    testY = dictTest[b'labels']

    score = testX.dot(w)

    ypredict = np.argmax(score, axis=1)
    

    return np.mean(ypredict == testY)

predict(w)


0.1005

In [23]:
def loss(x, y, reg, w):
    num_train = x.shape[0]
    score = x.dot(w)
    correct_score_class = score[range(num_train), list(y)].reshape(-1,1)
    margins = np.maximum(0, score - correct_score_class + delta)
    margins[range(num_train), list(y)] = 0
    loss = np.sum(margins) / num_train + 0.5 * reg * np.sum(w * w)

    num_classes = w.shape[1]
    gradient = np.zeros((num_train, num_classes))
    gradient[margins > 0] = 1
    gradient[range(num_train), list(y)] = 0
    gradient[range(num_train), list(y)] = -np.sum(gradient, axis=1)
    

    dw = (x.T).dot(gradient)
    dw = dw / num_train + reg * w
    return loss, dw


In [24]:
def train(x, y, w):
    loss_history = []
    for i in range(100):
        lossSum, grad = loss(x, y, lam, w)
        w -= learning_rate * grad
        loss_history.append(lossSum)
    print("loss = ", loss_history[-1])
    return loss_history
    pass

def runingTrain(time=10000):
    for i in range(50):
        
        start = 0
        numTrain = 200
        Xtrain = trainX[start:numTrain+start]
        trainY = dict[b'labels'][start:numTrain+start]
        start += numTrain
        
        train(Xtrain, trainY, w)
        
        print(predict(w))

runingTrain()

loss =  1439.8822830124657


0.1206


loss =  24.66470640559959


0.2387


loss =  0.6032479593635584


...

0.2365


loss =  0.00014945232522163473


0.2365


loss =  0.00014945229533117266


0.2365


In [90]:
#softmax multi-classification

def getLossAndDW(x, y, weight):
    LOSS = 0.0
    n = x.shape[0]
    score = x.dot(weight)
    correct_class = score[np.arange(n), y].reshape(n, 1)
    exp_sum = np.sum(np.exp(score), axis=1).reshape(n, 1)
    LOSS += np.sum(np.log(exp_sum) - correct_class)
    LOSS /= n
    LOSS += 0.5 * reg * np.sum(w ** 2)

    dweight = np.exp(score) / exp_sum
    dweight[range(n), list(y)] -= 1
    dweight = (x.T).dot(dweight) / n + reg * weight
    return LOSS, dweight


In [92]:
def train_softmax(weight, losses, acc,time=10):
    num_train = 200
    for k in range(time):
        for i in range(4):
            start = 0
            print("%d training data : " % (i + 1))
            for j in range(50):
                x = trainData[i][start:num_train + start]
                y = trainLabel[i][start:num_train + start]
                start += num_train
                loss, dw = getLossAndDW(x, y, weight)
                weight -= learning_rate * dw
                accr = predict(weight)
                if j % 5 == 0:
                   print("loss = ", loss, " ", accr)
                losses.append(loss)
                acc.append(accr)


In [95]:
dict1 = unpickle("CS231n/cifar-10-batches-py/data_batch_1")
dict2 = unpickle("CS231n/cifar-10-batches-py/data_batch_2")
dict3 = unpickle("CS231n/cifar-10-batches-py/data_batch_3")
dict4 = unpickle("CS231n/cifar-10-batches-py/data_batch_4")
dict5 = unpickle("CS231n/cifar-10-batches-py/data_batch_5")
classification = 10
Xtrain = dict[b'data']
Ytrain = dict[b'labels']
trainData = [dict1[b'data'], dict2[b'data'], dict3[b'data'], dict4[b'data'], dict5[b'data']]
trainLabel = [dict1[b'labels'], dict2[b'labels'], dict3[b'labels'], dict4[b'labels'], dict5[b'labels']]
reg = 1e-5
learning_rate = 1e-7
LOSS_list = []
acc = []
weight = 0.001 * np.random.randn(Xtrain.shape[1],classification)
train_softmax(weight, LOSS_list, acc, time=50)


1 training data : 


loss =  8.780300771392426   0.1111


loss =  6.843579517093624   0.1042


loss =  5.800097861027525   0.1052


loss =  5.051260209777784   0.1105


...

loss =  2.1287959493208746   0.3346


loss =  2.036436599730597   0.3334


loss =  1.9255997383856163   0.335


loss =  2.186601136787854   0.335


loss =  2.0670191310617376   0.3361


In [98]:
import matplotlib.pyplot as plt

x = np.linspace(1, len(LOSS_list)/100, len(LOSS_list)/100)

plt.plot(x, LOSS_list[::100])

plt.xlabel("training time")
plt.ylabel("training loss")

plt.show()

/Users/GreenArrow/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: object of type <class 'float'> cannot be safely interpreted as an integer.
  This is separate from the ipykernel package so we can avoid doing imports until


In [99]:
import matplotlib.pyplot as plt

x = np.linspace(1, len(LOSS_list)/100, len(LOSS_list)/100)

plt.plot(x, acc[::100])

plt.xlabel("training time")
plt.ylabel("training accuracy")

plt.show()

/Users/GreenArrow/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: object of type <class 'float'> cannot be safely interpreted as an integer.
  This is separate from the ipykernel package so we can avoid doing imports until
